# Chapter 9: CycleGAN
This Colab / Ipython notebook will walk you through: 
1. Install extra dependencies that Colab does not natively support
2. Download the apple2orange dataset using the `%%bash` magic.
3. Create the `DataLoader()` class that server as our core data structure
4. Do all the necessary imports
5. Define the CycleGAN in the following steps:

> 5.1 Define the key hyper-parameters and flow

> 5.2 Define the static methods we are going to be reusing in the generator and discriminator

> 5.3 Build the generator and the discriminator

> 5.4 Define the helper function for sampling our translated images

> 5.5 Define the explicit training loop

6. Fit the CycleGAN and talk about the results!


--- 

## Helper functions 
1. Pip installing directly from GitHub the `keras-contrib` folder, which we will need for the `InstanceNormalization`. We tested this git-hash version of the repository: 
`46fcdb9384b3bc9399c651b2b43640aa54098e64`
2. Downloading the dataset to our cloud-based—ephemeral—storage

In [ ]:
#@title
#!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [13]:
#@title
import os
# import requests

FILE='apple2orange'
#URL=https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/$FILE.zip #not valid anymore
# URL=r'https://www.kaggle.com/datasets/balraj98/apple2orange-dataset?resource=download'
# ZIP_FILE=rf'datasets/{FILE}.zip'
TARGET_DIR=rf'datasets/{FILE}'
print(f'{TARGET_DIR}')
#wget -N URL -O $ZIP_FILE
# response = requests.get(URL)

# # Open a file in binary write mode and write the content
# with open(ZIP_FILE, 'wb') as file:
#     file.write(response.content)

os.makedirs(TARGET_DIR, exist_ok=True)

#download from https://www.kaggle.com/datasets/balraj98/apple2orange-dataset?resource=download and manually extract and put into TARGET_DIR
#unzip $ZIP_FILE -d ./datasets/
#rm $ZIP_FILE


datasets/apple2orange


## DataLoader
We define our key data-holding object in the (hidden) cell below. With it, we can:
1. `load_data` from disk based on the dataset name, which we will define in `CycleGAN`'s `__init__`
2. `load_batch` during training—note this defined as a generator for greater efficiency
3. A helper functions called `imread`, which we use in `load_data`

In [2]:
#@title
import scipy
from glob import glob
import numpy as np
import cv2

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation=cv2.INTER_CUBIC)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation=cv2.INTER_CUBIC)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('datasets/%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('datasets/%s/%sB/*' % (self.dataset_name, data_type))
        #path_A = glob(os.path.join('datasets', self.dataset_name, f'{data_type}A', '*'))
        #path_B = glob(os.path.join('datasets', self.dataset_name, f'{data_type}B', '*'))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                #print(f'img_A: {img_A}')
                #print(f'img_B: {img_B}')
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                #img_A = scipy.misc.imresize(img_A, self.img_res)
                img_A = cv2.resize(img_A, self.img_res, interpolation=cv2.INTER_CUBIC)
                #img_B = scipy.misc.imresize(img_B, self.img_res)
                img_B = cv2.resize(img_B, self.img_res, interpolation=cv2.INTER_CUBIC)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def imread(self, path):
        #return scipy.misc.imread(path, mode='RGB').astype(np.float)
        # Read the image using OpenCV (default is BGR format)
        image_bgr = cv2.imread(path)

        # Check if the image was loaded successfully
        if image_bgr is None:
            raise ValueError("Error: Image not found or unable to load.")

        # Convert the BGR image to RGB
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        # Convert the image to float
        image_float = image_rgb.astype(np.float32)  # Use np.float32 for precision

        return image_float
    


## Good old imports

In [3]:
from __future__ import print_function, division
#import scipy
from tensorflow.keras.datasets import mnist
#from keras_contrib.layers.normalization import InstanceNormalization
from tensorflow.keras.layers import GroupNormalization
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
#from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.layers import LeakyReLU
#from tensorflow.keras.layers.convolutional import UpSampling2D, Conv2D
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

## CycleGAN itself

1. We define some parameters
2. We build the discriminators and compile them
3. We build the generators and compile them (these are a bit more complicated)

In [4]:
class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'apple2orange'
        # Use the DataLoader object to import a preprocessed dataset
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.9 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)
        
        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])
        self.d_B.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B,
                                       reconstr_A, reconstr_B,
                                       img_A_id, img_B_id])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              loss_weights=[1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id],
                              optimizer=optimizer)

## Static methods
Two things to pay attention to:
1. We are using a `CycleGAN` class that inherits from the `CycleGAN` class from the previous cell. As far as I know, this is the best way of defining a class across multiple cells for educational purposes. But I agree with Joel Grus—this is not the best way, but I still like notebooks.
2. We are using `@staticmethod`, which allows us to shrink the code size by further ~25 lines. For more on [static methods I found this SO answer to be quite good](https://stackoverflow.com/questions/136097/what-is-the-difference-between-staticmethod-and-classmethod). We will use these later to more compactly write both the generator and the discriminator. 

In [5]:
class CycleGAN(CycleGAN):
      @staticmethod
      def conv2d(layer_input, filters, f_size=4, normalization=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size,
                   strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            #d = InstanceNormalization()(d)
            d = GroupNormalization(groups=-1)(d)
        return d
      
      @staticmethod
      def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1,
                       padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            #u = InstanceNormalization()(u)
            u = GroupNormalization(groups=-1)(u)
            u = Concatenate()([u, skip_input])
            return u


## Generator

In [6]:
class CycleGAN(CycleGAN):
    def build_generator(self):
        """U-Net Generator"""
        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = self.conv2d(d0, self.gf)
        d2 = self.conv2d(d1, self.gf * 2)
        d3 = self.conv2d(d2, self.gf * 4)
        d4 = self.conv2d(d3, self.gf * 8)

        # Upsampling
        u1 = self.deconv2d(d4, d3, self.gf * 4)
        u2 = self.deconv2d(u1, d2, self.gf * 2)
        u3 = self.deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4,
                            strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

## Discriminator

In [7]:
class CycleGAN(CycleGAN):
    def build_discriminator(self):
      img = Input(shape=self.img_shape)

      d1 = self.conv2d(img, self.df, normalization=False)
      d2 = self.conv2d(d1, self.df * 2)
      d3 = self.conv2d(d2, self.df * 4)
      d4 = self.conv2d(d3, self.df * 8)

      validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

      return Model(img, validity)

## Sampling function
This is how we got the output in the Chapter. We need to run in special mode to get the translated images out, because now we are in "testing"/evaluation mode.

We then plot the whole cycle:

$ A -> \hat B -> \hat A$

As well as:

$ B -> \hat A -> \hat B$

In [8]:
class CycleGAN(CycleGAN):
      def sample_images(self, epoch, batch_i):
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)
        
        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        imgfolder=os.path.join("images",self.dataset_name)
        os.makedirs(imgfolder, exist_ok=True)
        imgfilename=f'{epoch}_{batch_i}.png'
        imgfilepath=os.path.join(imgfolder, imgfilename)
        print(imgfilepath)
        fig.savefig(imgfilepath)
        #plt.show()

        # Close the figure to free memory
        plt.close(fig)
        
        # Optionally, you can delete large variables if they are not needed further
        del imgs_A, imgs_B, fake_A, fake_B, reconstr_A, reconstr_B, gen_imgs

## Training loop

Here is where the real magic happens.



In [9]:
class CycleGAN(CycleGAN):
      def train(self, epochs, batch_size=1, sample_interval=50):
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)


        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                print(f'epoch: {epoch}, batch_i: {batch_i}')
                print(f'imgs_A.shape: {imgs_A.shape}')
                print(f'imgs_B.shape: {imgs_B.shape}')
                print(f'valid.shape: {valid.shape}')
                print(f'fake.shape: {fake.shape}')

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total discriminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                      [valid, valid,
                                                       imgs_A, imgs_B,
                                                       imgs_A, imgs_B])
                # If at save interval => plot the generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)

## Train the CycleGAN

Because we used the object-oriented design, we now need to instantiate the CycleGAN and run the `train`ing method.

In [10]:
cycle_gan = CycleGAN()
cycle_gan.train(epochs=100, batch_size=64, sample_interval=10)

c:\Users\hwenjun\.conda\envs\gan_in_actions\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


epoch: 0, batch_i: 0
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step


c:\Users\hwenjun\.conda\envs\gan_in_actions\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 915ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.00986135..0.7684972].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.009819269..1.017268].


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0051585436..0.98920417].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.048226893..1.068865].


images\apple2orange\5_0.png
epoch: 5, batch_i: 1
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 452ms/step
epoch: 5, batch_i: 2
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 418ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 408ms/step
epoch: 5, batch_i: 3
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 435ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 461ms/step
epoch: 5, batch_i: 4
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 460ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 459ms/step
epoch: 5, batch_i: 5
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8,

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.02829349..1.0089959].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.03159085..1.0330423].


images\apple2orange\6_0.png
epoch: 6, batch_i: 1
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 473ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 486ms/step
epoch: 6, batch_i: 2
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 497ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 475ms/step
epoch: 6, batch_i: 3
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 472ms/step
epoch: 6, batch_i: 4
imgs_A.shape: (64, 128, 128, 3)
imgs_B.shape: (64, 128, 128, 3)
valid.shape: (64, 8, 8, 1)
fake.shape: (64, 8, 8, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 471ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 458ms/step


: 

## Now we can look at the results!

You may have three observations:
1. The progress can be slow—even on the Colab GPU or on CPU.
2. The initial images can be quite bad and make no sense at all, but worry not—soon enough, the images will manage at least a decent reconstruction.
3. The later reconstructions are generally mixed: some look very convincing—such as the ones below—but some are merely an orange "repainting" of the apples and vice versa. Even the original authors of the paper mention this problem: CycleGAN is unable to substantially change the structure of the image, but only "restylize" it. So beware of this limitation.

![Good examples](https://i.ibb.co/n00BW1F/163-10.png)

Also, this is a good introduction to the reality of being a GAN researcher / practitioner. Most flashy results still tend to be cherry picked and most papers—e.g. [BigGAN being a notable exception](https://arxiv.org/abs/1809.11096)—do not share failure cases. BigGAN—the state of the art as of today (20/11/2018)—even [publishes Google folders worth of examples](https://drive.google.com/drive/folders/1SRYj8Ou1JZ4e09LqeawDHcUvoeY78tPO). 